# All PPTs

In [68]:
import nltk; nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /Users/Owner/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [69]:
!pip install --upgrade gensim
!pip install -U spacy
!pip install -U pip setuptools wheel
!pip install pyldavis

In [70]:
!python3 -m spacy download en_core_web_sm

     |████████████████████████████████| 13.9 MB 7.4 MB/s            
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [71]:
!pip install pyLDAvis==3.2.1

In [72]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

import gensim
import gensim.models
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

import spacy

import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [73]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [74]:
all = pd.read_csv('User Needs Assessments Data.csv')
all = all.loc[all['Person'] != 'Interviewer']

In [75]:
#tokenize words and clean text
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(all['Question/Answer']))

print(data_words[:1])

[['to', 'pay', 'monthly', 'utility', 'both', 'electricity', 'and', 'water']]


In [76]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['to', 'pay', 'monthly', 'utility', 'both', 'electricity', 'and', 'water']


In [77]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [78]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['pay', 'monthly', 'utility', 'electricity', 'water']]


In [79]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1)]]


In [80]:
id2word[0]

'electricity'

In [81]:
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('electricity', 1),
  ('monthly', 1),
  ('pay', 1),
  ('utility', 1),
  ('water', 1)]]

In [82]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [83]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.061*"take" + 0.041*"pay" + 0.041*"transfer" + 0.041*"start" + '
  '0.040*"process" + 0.040*"lengthy" + 0.021*"actually" + 0.021*"let" + '
  '0.021*"apartment" + 0.021*"year"'),
 (1,
  '0.003*"rude" + 0.003*"scale" + 0.003*"rating" + 0.003*"recall" + '
  '0.003*"reception" + 0.003*"number" + 0.003*"install" + 0.003*"instal" + '
  '0.003*"mother" + 0.003*"sometime"'),
 (2,
  '0.050*"form" + 0.025*"take" + 0.025*"tell" + 0.025*"come" + 0.025*"meter" + '
  '0.025*"actually" + 0.019*"reply" + 0.019*"send" + 0.019*"call" + '
  '0.019*"service"'),
 (3,
  '0.047*"take" + 0.037*"month" + 0.031*"get" + 0.030*"care" + 0.030*"mean" + '
  '0.029*"say" + 0.025*"end" + 0.025*"think" + 0.019*"form" + 0.019*"website"'),
 (4,
  '0.050*"really" + 0.033*"user" + 0.033*"function" + 0.033*"start" + '
  '0.033*"bot" + 0.033*"agent" + 0.017*"think" + 0.017*"care" + 0.017*"water" '
  '+ 0.017*"monthly"'),
 (5,
  '0.065*"bill" + 0.065*"receive" + 0.044*"issue" + 0.044*"laugh" + '
  '0.044*"term" + 0.0

In [84]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -6.254134450153429

Coherence Score:  0.5530412104641178


In [85]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary=lda_model.id2word, mds='mmds')   
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.251668 -0.037726       1        1  11.422969
3      0.282228  0.094134       2        1  11.382942
10    -0.221833 -0.171407       3        1   9.854579
17    -0.302915 -0.033168       4        1   8.070089
11     0.176819 -0.218900       5        1   7.903004
13    -0.144582 -0.185559       6        1   7.346151
18    -0.001218 -0.248792       7        1   5.566090
14     0.078713  0.235694       8        1   4.812142
16     0.114573 -0.047982       9        1   4.284106
9      0.011507 -0.168797      10        1   3.960632
6      0.153953 -0.118721      11        1   3.925985
4     -0.083733  0.248715      12        1   3.472530
12     0.148411  0.158086      13        1   3.427606
19    -0.117178  0.139208      14        1   3.348248
15    -0.029457  0.173694      15        1   2.978310
8      0.120677  0.044054      16        1   2.935153
0     -0.168543  0.010770      17        1   2.493465
5     -0.208030  0.104159      18        1   2.283532
7     -0.038665  0.012438      19        1   0.410541
1     -0.022394  0.010100      20        1   0.121926, topic_info=        Term     Freq      Total Category  logprob  loglift
20      form  32.0000  32.000000  Default  30.0000  30.0000
2        pay  20.0000  20.000000  Default  29.0000  29.0000
104     bill  15.0000  15.000000  Default  28.0000  28.0000
22   service  24.0000  24.000000  Default  27.0000  27.0000
110  account  10.0000  10.000000  Default  26.0000  26.0000
..       ...      ...        ...      ...      ...      ...
25       use   0.0038   8.570687  Topic20  -5.9738  -1.0115
26      care   0.0038   8.052896  Topic20  -5.9738  -0.9491
27       day   0.0038   5.612716  Topic20  -5.9738  -0.5881
28       end   0.0038   5.491418  Topic20  -5.9738  -0.5663
29     first   0.0038   6.173138  Topic20  -5.9738  -0.6833

[886 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
285       9  0.707267     able
365       7  0.683422   abroad
193       4  0.663283   access
110       3  0.278903  account
110       4  0.278903  account
...     ...       ...      ...
42        2  0.321344    wrong
42       10  0.321344    wrong
296       9  0.492340     year
296      17  0.492340     year
391       6  0.669418      yet

[835 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 4, 11, 18, 12, 14, 19, 15, 17, 10, 7, 5, 13, 20, 16, 9, 1, 6, 8, 2])

In [86]:
pyLDAvis.save_html(vis, 'All PPTs.html')

# PPT01

In [87]:
all = pd.read_csv('User Needs Assessments Data.csv')
PPT01 = all.loc[(all['Person'] == 'PPT01') & (all['Person'] != 'Interviewer')]

In [88]:
#tokenize words and clean text
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(PPT01['Question/Answer']))

print(data_words[:1])

[['to', 'pay', 'monthly', 'utility', 'both', 'electricity', 'and', 'water']]


In [89]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['to', 'pay', 'monthly', 'utility', 'both', 'electricity', 'and', 'water']


In [90]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [91]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['pay', 'monthly', 'utility', 'electricity', 'water']]


In [92]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1)]]


In [93]:
id2word[0]

'electricity'

In [94]:
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('electricity', 1),
  ('monthly', 1),
  ('pay', 1),
  ('utility', 1),
  ('water', 1)]]

In [95]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [96]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.006*"form" + 0.005*"take" + 0.005*"tell" + 0.005*"reply" + '
  '0.005*"service" + 0.005*"usually" + 0.005*"well" + 0.005*"print" + '
  '0.005*"fill" + 0.005*"send"'),
 (1,
  '0.077*"answer" + 0.077*"question" + 0.077*"satisfactory" + '
  '0.004*"available" + 0.004*"along" + 0.004*"bot" + 0.004*"chat" + '
  '0.004*"amount" + 0.004*"want" + 0.004*"actually"'),
 (2,
  '0.083*"time" + 0.082*"third" + 0.005*"customer" + 0.005*"ideally" + '
  '0.004*"form" + 0.004*"website" + 0.004*"people" + 0.004*"touchpoint" + '
  '0.004*"care" + 0.004*"chat"'),
 (3,
  '0.096*"provide" + 0.065*"authentication" + 0.065*"i" + 0.065*"login" + '
  '0.033*"datum" + 0.033*"safeguard" + 0.033*"factor" + 0.033*"d" + '
  '0.033*"ease" + 0.033*"website"'),
 (4,
  '0.041*"form" + 0.041*"take" + 0.040*"end" + 0.038*"month" + 0.038*"half" + '
  '0.021*"reply" + 0.021*"fill" + 0.021*"customer" + 0.021*"get" + '
  '0.021*"wrong"'),
 (5,
  '0.040*"people" + 0.040*"customer" + 0.030*"go" + 0.030*"website" + '
  

In [97]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -5.832702487707138

Coherence Score:  0.6421994319274502


In [98]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary=lda_model.id2word, mds='mmds')   
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5      0.218990  0.108026       1        1  19.043854
19    -0.028541 -0.287182       2        1  11.499027
10    -0.230914 -0.147278       3        1  10.749107
6     -0.189594  0.211609       4        1  10.641483
7      0.081580 -0.243161       5        1   8.433715
4      0.154578 -0.131113       6        1   7.781288
14     0.005090  0.260314       7        1   7.080249
9     -0.053044 -0.187962       8        1   4.860557
17    -0.204196 -0.006464       9        1   4.547862
3      0.133438  0.218968      10        1   4.344860
8      0.238151 -0.012176      11        1   4.224901
15    -0.172371  0.053948      12        1   3.933656
11    -0.034619 -0.024595      13        1   1.082299
1     -0.011583  0.063116      14        1   0.665575
2      0.039865  0.027536      15        1   0.468931
0      0.010335  0.018780      16        1   0.131835
13     0.010709  0.019408      17        1   0.127700
12     0.010709  0.019408      18        1   0.127700
16     0.010709  0.019408      19        1   0.127700
18     0.010709  0.019408      20        1   0.127700, topic_info=        Term       Freq      Total Category  logprob  loglift
2        pay  14.000000  14.000000  Default  30.0000  30.0000
20      form  16.000000  16.000000  Default  29.0000  29.0000
166  website   7.000000   7.000000  Default  28.0000  28.0000
7       time   4.000000   4.000000  Default  27.0000  27.0000
12      take   8.000000   8.000000  Default  26.0000  26.0000
..       ...        ...        ...      ...      ...      ...
25       use   0.002861   5.987620  Topic20  -5.3566  -0.9830
26      care   0.002861   4.490593  Topic20  -5.3566  -0.6953
27       day   0.002861   2.776508  Topic20  -5.3566  -0.2145
28       end   0.002861   3.408373  Topic20  -5.3566  -0.4195
29     first   0.002861   2.050421  Topic20  -5.3566   0.0887

[954 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
193      10  0.867947    access
110       2  0.735211   account
199       4  0.757158    actual
138       1  0.719930  actually
103       2  0.488818       add
...     ...       ...       ...
41        6  0.813318     whole
80        3  0.464819      work
80        4  0.464819      work
42        3  0.485313     wrong
42        6  0.485313     wrong

[349 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 20, 11, 7, 8, 5, 15, 10, 18, 4, 9, 16, 12, 2, 3, 1, 14, 13, 17, 19])

In [99]:
pyLDAvis.save_html(vis, 'PPT01.html')

# PPT02

In [100]:
all = pd.read_csv('User Needs Assessments Data.csv')
PPT02 = all.loc[(all['Person'] == 'PPT02') & (all['Person'] != 'Interviewer')]

In [101]:
#tokenize words and clean text
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(PPT02['Question/Answer']))

print(data_words[:1])

[['to', 'pay', 'my', 'bills', 'and', 'to', 'get', 'an', 'estimate', 'of', 'my', 'upcoming', 'bills', 'so', 'that', 'can', 'put', 'in', 'the', 'inaudible', 'meter', 'readings', 'and', 'get', 'an', 'estimate', 'of', 'my', 'monthly', 'bills', 'that', 'and', 'to', 'check', 'whether', 'there', 'are', 'any', 'interruptions', 'in', 'the', 'services']]


In [102]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['to', 'pay', 'my', 'bills', 'and', 'to', 'get', 'an', 'estimate', 'of', 'my', 'upcoming', 'bills', 'so', 'that', 'can', 'put', 'in', 'the', 'inaudible', 'meter', 'readings', 'and', 'get', 'an', 'estimate', 'of', 'my', 'monthly', 'bills', 'that', 'and', 'to', 'check', 'whether', 'there', 'are', 'any', 'interruptions', 'in', 'the', 'services']


In [103]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [104]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['pay', 'bill', 'get', 'estimate', 'upcoming', 'bill', 'put', 'inaudible', 'meter', 'reading', 'get', 'estimate', 'monthly', 'bill', 'check', 'interruption', 'service']]


In [105]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 3), (1, 1), (2, 2), (3, 2), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1)]]


In [106]:
id2word[0]

'bill'

In [107]:
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('bill', 3),
  ('check', 1),
  ('estimate', 2),
  ('get', 2),
  ('inaudible', 1),
  ('interruption', 1),
  ('meter', 1),
  ('monthly', 1),
  ('pay', 1),
  ('put', 1),
  ('reading', 1),
  ('service', 1),
  ('upcoming', 1)]]

In [108]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [109]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.006*"phonecall" + 0.006*"receipt" + 0.006*"come" + 0.006*"expect" + '
  '0.006*"good" + 0.006*"government" + 0.006*"happen" + 0.006*"inform" + '
  '0.006*"installation" + 0.006*"least"'),
 (1,
  '0.095*"bill" + 0.064*"pay" + 0.064*"definitely" + 0.033*"service" + '
  '0.033*"find" + 0.033*"respect" + 0.033*"provider" + 0.033*"mean" + '
  '0.033*"really" + 0.033*"choose"'),
 (2,
  '0.006*"phonecall" + 0.006*"receipt" + 0.006*"come" + 0.006*"expect" + '
  '0.006*"good" + 0.006*"government" + 0.006*"happen" + 0.006*"inform" + '
  '0.006*"installation" + 0.006*"least"'),
 (3,
  '0.074*"mean" + 0.050*"inaudible" + 0.050*"meter" + 0.050*"long" + '
  '0.050*"install" + 0.050*"ideally" + 0.026*"call" + 0.026*"inform" + '
  '0.026*"come" + 0.026*"time"'),
 (4,
  '0.039*"actually" + 0.039*"round" + 0.039*"come" + 0.039*"today" + '
  '0.039*"know" + 0.039*"inform" + 0.039*"call" + 0.020*"service" + '
  '0.020*"inaudible" + 0.020*"good"'),
 (5,
  '0.145*"service" + 0.098*"government" + 0

In [110]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -5.7467466364929445

Coherence Score:  0.6591601944623011


In [111]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary=lda_model.id2word, mds='mmds')   
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
18    -0.236923 -0.063010       1        1  23.070788
17    -0.247229  0.104048       2        1  16.072787
16     0.264315 -0.016205       3        1  11.925567
4     -0.035681 -0.243058       4        1  10.245880
15     0.199178  0.178414       5        1   8.148476
9      0.040272  0.230238       6        1   7.336999
3     -0.104178  0.210447       7        1   7.277665
1     -0.135098 -0.161247       8        1   5.240117
6      0.081001 -0.206017       9        1   3.949858
5      0.132999 -0.108351      10        1   2.786350
19     0.133968  0.069299      11        1   2.558860
10    -0.028198  0.032008      12        1   0.484529
12    -0.008294 -0.003631      13        1   0.116163
13    -0.007609 -0.003000      14        1   0.116044
14    -0.008087 -0.003323      15        1   0.111653
11    -0.008087 -0.003323      16        1   0.111653
8     -0.008087 -0.003323      17        1   0.111653
7     -0.008087 -0.003323      18        1   0.111653
2     -0.008087 -0.003323      19        1   0.111653
0     -0.008087 -0.003323      20        1   0.111653, topic_info=        Term       Freq      Total Category  logprob  loglift
11   service  12.000000  12.000000  Default   30.000  30.0000
14     think   6.000000   6.000000  Default   29.000  29.0000
28      form  11.000000  11.000000  Default   28.000  28.0000
0       bill   5.000000   5.000000  Default   27.000  27.0000
27      fill   7.000000   7.000000  Default   26.000  26.0000
..       ...        ...        ...      ...      ...      ...
23  customer   0.002661   6.599535  Topic20   -5.193  -1.0185
24  download   0.002661   2.819793  Topic20   -5.193  -0.1682
26     email   0.002661   2.858122  Topic20   -5.193  -0.1817
27      fill   0.002661   7.027357  Topic20   -5.193  -1.0813
28      form   0.002661  11.121791  Topic20   -5.193  -1.5404

[991 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
141       3  0.778761         able
49        1  0.593420      account
49        2  0.445065      account
115       4  0.785074  acknowledge
159       1  0.739910       actual
...     ...       ...          ...
44        2  0.284943         well
44        3  0.284943         well
44        5  0.284943         well
44       11  0.284943         well
158       3  0.778744         year

[310 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[19, 18, 17, 5, 16, 10, 4, 2, 7, 6, 20, 11, 13, 14, 15, 12, 9, 8, 3, 1])

In [112]:
pyLDAvis.save_html(vis, 'PPT02.html')

# PPT03

In [113]:
all = pd.read_csv('User Needs Assessments Data.csv')
PPT03 = all.loc[(all['Person'] == 'PPT03') & (all['Person'] != 'Interviewer')]

In [114]:
#tokenize words and clean text
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(PPT03['Question/Answer']))

print(data_words[:1])

[['laughs', 'because', 'it', 'the', 'only', 'place', 'to', 'do', 'so', 'no', 'in', 'malta', 'no', 'there', 'no', 'other', 'form', 'of', 'receiving', 'water', 'and', 'electricity', 'through', 'different', 'ehm', 'correct', 'me', 'if', 'wrong', 'but', 'it', 'the', 'only', 'solution', 'to', 'doing', 'so']]


In [115]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['laughs', 'because', 'it', 'the', 'only', 'place', 'to', 'do', 'so', 'no', 'in', 'malta', 'no', 'there', 'no', 'other', 'form', 'of', 'receiving', 'water', 'and', 'electricity', 'through', 'different', 'ehm', 'correct', 'me', 'if', 'wrong', 'but', 'it', 'the', 'only', 'solution', 'to', 'doing', 'so']


In [116]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [117]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['laugh', 'place', 'form', 'receive', 'water', 'electricity', 'different', 'ehm', 'correct', 'wrong', 'solution']]


In [118]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1)]]


In [119]:
id2word[0]

'correct'

In [120]:
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('correct', 1),
  ('different', 1),
  ('ehm', 1),
  ('electricity', 1),
  ('form', 1),
  ('laugh', 1),
  ('place', 1),
  ('receive', 1),
  ('solution', 1),
  ('water', 1),
  ('wrong', 1)]]

In [121]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [122]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.093*"complain" + 0.093*"really" + 0.047*"ask" + 0.047*"care" + '
  '0.047*"chase" + 0.047*"honest" + 0.047*"customer" + 0.047*"situation" + '
  '0.047*"talk" + 0.047*"always"'),
 (1,
  '0.087*"receive" + 0.087*"bill" + 0.058*"laugh" + 0.058*"issue" + '
  '0.058*"term" + 0.030*"online" + 0.030*"encounter" + 0.030*"home" + '
  '0.030*"high" + 0.030*"pay"'),
 (2,
  '0.006*"argue" + 0.006*"experience" + 0.006*"term" + 0.006*"typical" + '
  '0.006*"ui" + 0.006*"user" + 0.006*"whatnot" + 0.006*"abroad" + '
  '0.006*"apply" + 0.006*"set"'),
 (3,
  '0.080*"choice" + 0.080*"application" + 0.080*"form" + 0.080*"laugh" + '
  '0.080*"unfortunately" + 0.004*"user" + 0.004*"term" + 0.004*"whatnot" + '
  '0.004*"apply" + 0.004*"ui"'),
 (4,
  '0.097*"email" + 0.050*"straightforward" + 0.050*"send" + 0.050*"quite" + '
  '0.050*"basically" + 0.050*"download" + 0.050*"form" + 0.050*"sale" + '
  '0.050*"team" + 0.050*"sense"'),
 (5,
  '0.084*"take" + 0.057*"start" + 0.057*"pay" + 0.057*"transfer

In [123]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -5.744741023143876

Coherence Score:  0.5797543333497194


In [124]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary=lda_model.id2word, mds='mmds')   
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7     -0.077758  0.216608       1        1  21.005426
19     0.113019 -0.258276       2        1   9.004909
9      0.168558  0.214261       3        1   8.700253
5     -0.149589 -0.236244       4        1   8.693481
1     -0.213122  0.166189       5        1   8.381844
11     0.054655  0.229562       6        1   7.447658
17     0.225366 -0.085727       7        1   7.182982
13    -0.229060  0.035726       8        1   6.526928
16     0.217674  0.083485       9        1   6.202480
0     -0.192091 -0.052343      10        1   4.335293
8     -0.118844 -0.141088      11        1   4.335292
4      0.123162 -0.153857      12        1   4.024286
18    -0.072853  0.028284      13        1   1.852140
3      0.092732 -0.030585      14        1   1.543683
6      0.009692 -0.002666      15        1   0.127224
12     0.009692 -0.002665      16        1   0.127224
14     0.009692 -0.002665      17        1   0.127224
15     0.009692 -0.002666      18        1   0.127224
2      0.009692 -0.002666      19        1   0.127224
10     0.009692 -0.002666      20        1   0.127224, topic_info=         Term      Freq     Total Category  logprob  loglift
4        form  5.000000  5.000000  Default  30.0000  30.0000
5       laugh  4.000000  4.000000  Default  29.0000  29.0000
89     really  6.000000  6.000000  Default  28.0000  28.0000
34       take  5.000000  5.000000  Default  27.0000  27.0000
11       bill  4.000000  4.000000  Default  26.0000  26.0000
..        ...       ...       ...      ...      ...      ...
25  initially  0.002505  2.149619  Topic20  -5.0938  -0.0876
26       last  0.002505  1.263124  Topic20  -5.0938   0.4441
27    lengthy  0.002505  1.975623  Topic20  -5.0938  -0.0032
28        let  0.002505  4.345367  Topic20  -5.0938  -0.7914
29       mean  0.002505  2.569756  Topic20  -5.0938  -0.2661

[972 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
110       1  0.725570    abroad
17        1  0.831815   account
17        4  0.207954   account
18        4  0.791688  actually
38        1  0.725961   already
...     ...       ...       ...
109       8  0.277409   whatnot
147       5  0.794306      work
10        7  0.827573     wrong
37        4  0.791688      year
159       3  0.792795       yet

[258 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[8, 20, 10, 6, 2, 12, 18, 14, 17, 1, 9, 5, 19, 4, 7, 13, 15, 16, 3, 11])

In [125]:
pyLDAvis.save_html(vis, 'PPT03.html')